<a href="https://colab.research.google.com/github/kobe4cn/modelscope_master/blob/main/%E5%9B%BE%E5%83%8F%E8%AF%86%E5%88%AB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install opencv_python
!pip install tf_slim
!pip install modelscope
!pip install pyperclip
!pip install pyclipper
!pip install openpyxl
!pip install xlwings

In [2]:
!pip install transformers

In [3]:
from modelscope.pipelines import pipeline
from modelscope.utils.constant import Tasks
import numpy as np
import cv2
import math
import pyclipper


2023-07-14 05:26:00,916 - modelscope - INFO - PyTorch version 2.0.1+cu118 Found.
2023-07-14 05:26:00,921 - modelscope - INFO - TensorFlow version 2.12.0 Found.
2023-07-14 05:26:00,925 - modelscope - INFO - Loading ast index from /root/.cache/modelscope/ast_indexer
2023-07-14 05:26:00,970 - modelscope - INFO - Loading done! Current index file version is 1.7.1, with md5 4b833443e0cc3ba8e5d71d367ec84212 and a total number of 861 components indexed


In [4]:
# 在图片上定位文本位置
def crop_image(img, position):
    def distance(x1,y1,x2,y2):
        return math.sqrt(pow(x1 - x2, 2) + pow(y1 - y2, 2))
    position = position.tolist()
    for i in range(4):
        for j in range(i+1, 4):
            if(position[i][0] > position[j][0]):
                tmp = position[j]
                position[j] = position[i]
                position[i] = tmp
    if position[0][1] > position[1][1]:
        tmp = position[0]
        position[0] = position[1]
        position[1] = tmp

    if position[2][1] > position[3][1]:
        tmp = position[2]
        position[2] = position[3]
        position[3] = tmp

    x1, y1 = position[0][0], position[0][1]
    x2, y2 = position[2][0], position[2][1]
    x3, y3 = position[3][0], position[3][1]
    x4, y4 = position[1][0], position[1][1]

    corners = np.zeros((4,2), np.float32)
    corners[0] = [x1, y1]
    corners[1] = [x2, y2]
    corners[2] = [x4, y4]
    corners[3] = [x3, y3]

    img_width = distance((x1+x4)/2, (y1+y4)/2, (x2+x3)/2, (y2+y3)/2)
    img_height = distance((x1+x2)/2, (y1+y2)/2, (x4+x3)/2, (y4+y3)/2)

    corners_trans = np.zeros((4,2), np.float32)
    corners_trans[0] = [0, 0]
    corners_trans[1] = [img_width - 1, 0]
    corners_trans[2] = [0, img_height - 1]
    corners_trans[3] = [img_width - 1, img_height - 1]

    transform = cv2.getPerspectiveTransform(corners, corners_trans)
    dst = cv2.warpPerspective(img, transform, (int(img_width), int(img_height)))
    return dst

In [5]:
def order_point(coor):
    arr = np.array(coor).reshape([4, 2])
    sum_ = np.sum(arr, 0)
    centroid = sum_ / arr.shape[0]
    theta = np.arctan2(arr[:, 1] - centroid[1], arr[:, 0] - centroid[0])
    sort_points = arr[np.argsort(theta)]
    sort_points = sort_points.reshape([4, -1])
    if sort_points[0][0] > centroid[0]:
        sort_points = np.concatenate([sort_points[3:], sort_points[:3]])
    sort_points = sort_points.reshape([4, 2]).astype('float32')
    return sort_points

In [6]:
#先检测图片中的文本位置
ocr_detection = pipeline(Tasks.ocr_detection, model='damo/cv_resnet18_ocr-detection-line-level_damo')
# ocr_detection = pipeline(Tasks.ocr_detection, model='damo/cv_resnet18_ocr-detection-line-level_damo')
#ocr_detection = pipeline(Tasks.ocr_detection, model='damo/cv_resnet18_ocr-detection-db-line-level_damo')
#基于文本位置，识别文本内容信息
ocr_recognition = pipeline(Tasks.ocr_recognition, model='damo/cv_convnextTiny_ocr-recognition-general_damo')
#文本属性识别
ner_pipeline = pipeline(Tasks.named_entity_recognition, 'damo/nlp_raner_named-entity-recognition_chinese-base-ecom-50cls')


2023-07-14 05:26:03,581 - modelscope - INFO - Model revision not specified, use the latest revision: v1.0.0
2023-07-14 05:26:18,765 - modelscope - INFO - initiate model from /root/.cache/modelscope/hub/damo/cv_resnet18_ocr-detection-line-level_damo
2023-07-14 05:26:18,768 - modelscope - INFO - initiate model from location /root/.cache/modelscope/hub/damo/cv_resnet18_ocr-detection-line-level_damo.
2023-07-14 05:26:18,775 - modelscope - WARNING - No preprocessor field found in cfg.
2023-07-14 05:26:18,777 - modelscope - WARNING - No val key and type key found in preprocessor domain of configuration.json file.
2023-07-14 05:26:18,779 - modelscope - WARNING - Cannot find available config to build preprocessor at mode inference, current config: {'model_dir': '/root/.cache/modelscope/hub/damo/cv_resnet18_ocr-detection-line-level_damo'}. trying to build by task and model information.
2023-07-14 05:26:18,780 - modelscope - WARNING - Find task: ocr-detection, model type: None. Insufficient info

In [7]:
import cv2
import numpy as np
#将文本信息绘制回图片
def draw_boxes_on_image(image_path, output_path, box_coordinates,label):

# 读取图片
    image = cv2.imread(image_path)

    # 将框的坐标绘制到图片上
    pts = np.array(box_coordinates, np.int32)
    pts = pts.reshape((-1, 1, 2))
    cv2.polylines(image, [pts], isClosed=True, color=(0, 0, 255), thickness=1)  # 修改线宽为1

    # 在每个框的位置上增加序号标记
    label = label
    #cv2.putText(image, str(label), (int(box_coordinates[0]), int(box_coordinates[1])), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 1)

    cv2.putText(image, str(label), (int(box_coordinates[0]), int(box_coordinates[1])),cv2.FONT_HERSHEY_SIMPLEX | cv2.FONT_ITALIC, 0.6, (0, 0, 255), 2)


    # 保存修改后的图片
    cv2.imwrite(output_path, image)

In [8]:
import os
#遍历目录读取图片
def get_all_file_paths(directory):
    file_paths = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            file_path = os.path.join(root, file)
            file_paths.append(os.path.abspath(file_path))
    return file_paths

# 示例用法
# directory = "/content/drive/MyDrive/Colab Notebooks/modelscope/images"
# file_paths = get_all_file_paths(directory)
# print(file_paths)

In [9]:
#增加输出后缀
import os

def add_out_suffix(file_path):
    dir_name = os.path.dirname(file_path)  # 获取文件所在目录路径
    file_name = os.path.basename(file_path)  # 获取文件名称
    file_name, extension = os.path.splitext(file_name)  # 将文件名称和扩展名拆分开

    # 在文件所在路径的同级目录下创建一个名为 "out" 的文件夹
    out_dir = os.path.join(os.path.dirname(dir_name), "out")
    os.makedirs(out_dir, exist_ok=True)  # 创建 "out" 文件夹，如果已存在则不进行任何操作

    # 在文件名称中添加 "_out" 后缀
    new_file_name = file_name + "_out" + extension

    # 构建新的文件路径
    new_file_path = os.path.join(out_dir, new_file_name)

    return new_file_path
add_out_suffix('/content/drive/MyDrive/Colab Notebooks/modelscope/images/lamer.jpg')

'/content/drive/MyDrive/Colab Notebooks/modelscope/out/lamer_out.jpg'

In [10]:
#将解析到图片信息分行存储
def save_text_to_string(saved_text,text):

    saved_text += text + "\n"  # 将每行文本追加到字符串，并添加换行符

    return saved_text

In [11]:
#输出的图片和抽取的图片文字内容转换成excel输出

import os
import pandas as pd
import openpyxl
from openpyxl import Workbook
from openpyxl.drawing.image import Image

def append_to_excel(image_path, text, output_file):
    output_dir = os.path.dirname(output_file)  # 获取输出文件所在目录路径
    os.makedirs(output_dir, exist_ok=True)  # 创建输出目录，如果已存在则不进行任何操作

    # 检查输出文件是否存在
    file_exists = os.path.isfile(output_file)

    # 加载Excel文件或创建新的工作簿和工作表
    if file_exists:
        wb = Workbook()
        wb = openpyxl.load_workbook(output_file)
        sheet = wb.active
    else:
        wb = Workbook()
        sheet = wb.active
        sheet.append(['Image', 'Text','Product'])

    # 插入图片路径和文本信息到新行
    row = sheet.max_row + 1
    sheet['B' + str(row)].value = text
    sheet.column_dimensions['B'].width = 80

    # 插入图片到指定位置
    img = Image(image_path)
    sheet.add_image(img, 'A{}'.format(row))

    # 缩小图片尺寸
    img_width = img.width
    img_height = img.height
    # 调整单元格大小以适应缩小后的图片尺寸
    sheet.column_dimensions['A'].width = 80 #img.width / 6  # 调整列宽
    sheet.row_dimensions[row].height = img.height / 1.5  # 调整行高
    img.width = int(img_width * 0.8)
    img.height = int(img_height * 0.8)

    # # 调整单元格大小以适应缩小后的图片尺寸
    # sheet.column_dimensions['A'].width = img.width / 6  # 调整列宽
    # sheet.row_dimensions[row].height = img.height / 1.5  # 调整行高

    # 保存Excel文件
    wb.save(output_file)




In [12]:
import datetime
#日期格式文件名生成
def generate_file_name_with_date(extension):
    current_datetime = datetime.datetime.now()
    formatted_date = current_datetime.strftime("%Y-%m-%d")
    file_name = f"output_{formatted_date}.{extension}"
    return file_name

In [13]:
#产品数据输出

import os
import pandas as pd
import openpyxl
from openpyxl import Workbook
from openpyxl.drawing.image import Image

def append_product_excel(text, output_file):
    output_dir = os.path.dirname(output_file)  # 获取输出文件所在目录路径
    os.makedirs(output_dir, exist_ok=True)  # 创建输出目录，如果已存在则不进行任何操作

    # 检查输出文件是否存在
    file_exists = os.path.isfile(output_file)

    # 加载Excel文件或创建新的工作簿和工作表
    if file_exists:
        wb = Workbook()
        wb = openpyxl.load_workbook(output_file)
        sheet = wb.active
    else:
        wb = Workbook()
        sheet = wb.active
        sheet.append(['Image', 'Text','Product'])

    # 插入图片路径和文本信息到新行
    row = sheet.max_row
    sheet['C' + str(row)].value = text
    print(text)


    # 保存Excel文件
    wb.save(output_file)

In [14]:


import numpy as np

def box_center(box):
    """计算框的中心点坐标"""
    x = np.mean(box[::2])
    y = np.mean(box[1::2])
    return x, y

#找尺寸相关标签的最近列
def find_closest_above(boxes, target_box):
    """找出在给定框正上方且距离最近的框"""
    target_center = box_center(target_box)
    closest_box = None
    closest_center_dist = np.inf
    for box in boxes:
        center_x, center_y = box_center(box)
        if center_y < target_center[1]:  # 如果框在给定框的正上方
            dist = np.sqrt((center_x - target_center[0])**2 + (center_y - target_center[1])**2)  # 计算欧氏距离
            if dist < closest_center_dist:
                closest_center_dist = dist
                closest_box = box
    return closest_box



# 假设你的框在数组boxes中，你想找到在target_box上方的框
# boxes = np.array([472, 659, 524, 659, 524, 674, 472, 674])  # 你的框数组
# boxes = np.array([
#     [137, 649, 213, 648, 214, 657, 138, 658],
#     [473, 634, 525, 634, 525, 652, 473, 653],
#     [276, 662, 400, 662, 400, 684, 276, 684]

# ])
# target_box = np.array([474.0,659.0,524.0,659.0,524.0,673.0,474.0,673.0])  # 你的目标框
# closest_box = find_closest_above(boxes, target_box)
# print(closest_box)
# string = ', '.join(map(str, closest_box))
# print(dict_data[string])


In [15]:
result= ner_pipeline('5.2620点锁定星品好礼')
print(result)

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:884: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


{'output': []}


In [16]:
#解析所有商品增加进商品列
def find_product(output_file):
  # print(len(product_arr))
  # print(dict_data)
  for pro in product_arr:
    closest_box = find_closest_above(det_result, pro)
    # print(closest_box)
    close_str= ', '.join(map(str, closest_box))
    pro_int = [int(x) for x in pro]
    pro_str = ', '.join(map(str, pro_int))
    # print(pro_str)
    # print(dict_data[close_str])
    # print(dict_data[pro_str])
    dict_data[close_str]=dict_data[close_str]+dict_data[pro_str]
  # print(dict_data)
  str_result=''
  for value in dict_data.values():
    print(value)

    # result = None
    if value!='':
      result= ner_pipeline(value)
    # if value=='5.2620点锁定星品好礼':
    else:
      continue
    check=0
    for item in result['output']:
      if '产品_核心产品' in item["type"]:
        check+=1
      if '尺寸规格' in item["type"]:
        check+=1
      if check>=2:
        str_result+=value+'\n'
        break
  append_product_excel(str_result,output_file)
  print(str_result,output_file)
  # find_product(file_out)

In [17]:
#获取目录下的所有需要解析的图片
image_files=get_all_file_paths('/content/drive/MyDrive/Colab Notebooks/modelscope/images')
#图片中解析的文本信息存储在这个字段

#文件输出路径
file_out='/content/drive/MyDrive/Colab Notebooks/modelscope/excel/'+""+generate_file_name_with_date('xlsx')
for img_path in image_files:
  text_parsing=""
  #图片路径或者提供URL
  img_path = img_path
  #转换未来图片的输出路径
  out_path = add_out_suffix(img_path)
  image_full = cv2.imread(img_path)
  #图片输出路径
  cv2.imwrite(out_path, image_full)
  # #也可以提供URL
  # img_url='https://modelscope.oss-cn-beijing.aliyuncs.com/test/images/ocr_detection.jpg'

  #图片位置
  det_result = ocr_detection(image_full)

  det_result = det_result['polygons']
  dict_data={}
  product_arr=[]
  for i in range(det_result.shape[0]):
      pts = order_point(det_result[i])
      #图片位置
      image_crop = crop_image(image_full, pts)
      result = ocr_recognition(image_crop)
      #print(list(pts.reshape(-1)))
      # 读取图片
      image = cv2.imread(out_path)

      # 定义文本信息和框的坐标
      box_coordinates = list(pts.reshape(-1))

      # 将文本信息和框的坐标绘制到图片上
      draw_boxes_on_image(out_path, out_path, box_coordinates,i+1)

      print("box: %s" % ', '.join([str(int(e)) for e in list(pts.reshape(-1))]))

      print("text: %s. %s" % ((i+1),result['text']))

      #识别出来的文本内容
      word_content="".join(result['text'])
      #框的位置
      frame_location=', '.join([str(int(e)) for e in list(pts.reshape(-1))])
      #识别文本属性
      # result = ner_pipeline('云绒霜')
      if word_content !='':
        result= ner_pipeline(word_content)
        #print(result)
        if len(result['output'])==1 and '尺寸规格' in result['output'][0]['type']:
          product_arr.append(list(pts.reshape(-1)))
      dict_data[frame_location]=word_content
      text_parsing = save_text_to_string(text_parsing, str(i+1) + "." + word_content)
     #text_parsing=save_text_to_string(text_parsing,str(i+1)+"."+result['text'])

  append_to_excel(out_path,text_parsing,file_out)
  find_product(file_out)



box: 69, 781, 726, 783, 726, 795, 69, 792
text: 1. ['会员满颜礼需成为品牌会员后加入购物车领取,单笔订单满1000元限享1次加入会员后首次购买正装加赠,需先在品牌会员中心申领,每个ID限享I次']
box: 246, 185, 602, 186, 601, 223, 246, 223
text: 2. ['每蓝之谜修护精萃水']
box: 201, 375, 502, 375, 502, 424, 201, 424
text: 3. ['3D.COM京']
box: 431, 108, 691, 107, 691, 165, 432, 167
text: 4. ['淡纹焕亮']
box: 148, 108, 404, 108, 404, 166, 148, 166
text: 5. ['精华入水']
box: 26, 26, 222, 26, 222, 66, 26, 66
text: 6. ['LAMER']
box: 499, 379, 624, 379, 623, 410, 499, 409
text: 7. ['仲夏焕颜']
box: 276, 663, 399, 663, 398, 683, 276, 683
text: 8. ['精萃水150ml']
box: 586, 281, 687, 281, 687, 302, 586, 302
text: 9. ['会员专属礼']
box: 663, 503, 757, 503, 757, 517, 663, 517
text: 10. ['【先入会再购物】']
box: 136, 488, 225, 488, 225, 509, 136, 509
text: 11. ['LA MER']
box: 588, 307, 676, 307, 676, 321, 588, 321
text: 12. ['【先申领再购物】']
box: 589, 326, 675, 326, 675, 341, 589, 341
text: 13. ['云绒霜3.5ml']
box: 667, 474, 749, 474, 749, 489, 667, 489
text: 14. ['会员满额礼']
box: 137, 649, 213, 648, 214, 657, 138, 6

*加入产品的新列*
